# `sentinelsat` Python librairy
Tutorial to learn how to make use of the Python librairy `sentinelsat`, following explanation from the blog [acgeospatial.co.uk](http://www.acgeospatial.co.uk/sentinelsat_demo/).  
Documentation is also available [here](https://sentinelsat.readthedocs.io/en/stable/api.html#search-sentinel-2-by-tile).

`sentinelsat` makes you able to search for and download Sentinel2 data.

### Requirements

**Area of interest**
The area of interest is the area where we would like to find satellite data. It is defined by a geojson file. In this tutorial, we focus on the city Toulouse, in France. The file `toulouse.geojson`, in folder `data`, described the city borders.

**Librairies requirements**
* `sentinelsat`
* `folium` to display images footprint on interactive maps
* `geopandas` for spatial data manipulation
* `os` for sytem interactions
* `datetime` to work with data objects
* `zipfile` to work with ZIP archives

In [ ]:
import sentinelsat as sat
import folium
import geopandas as gpd
import os
from datetime import date
import zipfile

## Display area of interest

In [4]:
m = folium.Map(
    location=[43.6009,1.4332], 
    tiles='cartodbpositron',
    zoom_start=12
    )

area_of_interest = 'data/toulouse.geojson'

folium.GeoJson(
    area_of_interest
).add_to(m)

m

Notice how the city borders have been simplified. **Sentinelsat API only accepts simple geometry input because of limitation on request lenght.**

## Connect to Sentinel API
You need to be registered on [SciHub, Copernicus Open Access Hub](https://scihub.copernicus.eu/dhus/#/home) to be able to reach Sentinel API.

In [ ]:
# Change the following variables with your own login
user = 'username' 
password = 'password'

api = sat.SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

Convert area of interest from geojson to WKT to request the Sentinel API

In [4]:
footprint = sat.geojson_to_wkt(sat.read_geojson(area_of_interest))
print(footprint)

MULTIPOLYGON(((-0.4016 43.8941,2.2088 43.8941,2.2088 42.5674,-0.4016 42.5674,-0.4016 43.8941)))


**Query**

In [ ]:
products = api.query(footprint,
                     date = (date(2019, 2, 25), '20190227'),  # Notice the two ways of formating dates
                     platformname = 'Sentinel-2',             # Many other criteria are available for searching, have a look at the docs.
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0, 20))

The query returns an ordered dictionnary. To ease the reading, you can convert it into a geodataframe:

In [6]:
products_gdf = api.to_geodataframe(products)
products_gdf

,title,link,link_alternative,link_icon,summary,ingestiondate,beginposition,endposition,orbitnumber,relativeorbitnumber,...,platformidentifier,orbitdirection,platformserialidentifier,processingbaseline,processinglevel,producttype,platformname,size,uuid,geometry
1808663c-5c2c-43df-8e5e-640b045bcb2d,S2A_MSIL2A_20190225T105021_N0211_R051_T31TDH_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-02-25T10:50:21.024Z, Instrument: MS...",2019-02-25 23:31:27.320,2019-02-25 10:50:21.024,2019-02-25 10:50:21.024,19210,51,...,2015-028A,DESCENDING,Sentinel-2A,02.11,Level-2A,S2MSI2A,Sentinel-2,783.30 MB,1808663c-5c2c-43df-8e5e-640b045bcb2d,"POLYGON ((2.765516285000941 43.35106361101648,..."
2cfe1f72-2a7f-4d9b-ae16-b9dc171a6e90,S2A_MSIL2A_20190225T105021_N0211_R051_T30TYN_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-02-25T10:50:21.024Z, Instrument: MS...",2019-02-25 23:31:43.890,2019-02-25 10:50:21.024,2019-02-25 10:50:21.024,19210,51,...,2015-028A,DESCENDING,Sentinel-2A,02.11,Level-2A,S2MSI2A,Sentinel-2,1.15 GB,2cfe1f72-2a7f-4d9b-ae16-b9dc171a6e90,"POLYGON ((-0.5337219 43.32625697177811, 0.8181..."
74830722-4940-45c1-8ae3-c92389e9c51f,S2A_MSIL2A_20190225T105021_N0211_R051_T31TDJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-02-25T10:50:21.024Z, Instrument: MS...",2019-02-25 23:45:45.127,2019-02-25 10:50:21.024,2019-02-25 10:50:21.024,19210,51,...,2015-028A,DESCENDING,Sentinel-2A,02.11,Level-2A,S2MSI2A,Sentinel-2,1009.01 MB,74830722-4940-45c1-8ae3-c92389e9c51f,"POLYGON ((3.119621817797342 44.25333912070032,..."
aee954a6-584d-43dd-b21a-6add2e7b1826,S2A_MSIL2A_20190225T105021_N0211_R051_T30TYP_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-02-25T10:50:21.024Z, Instrument: MS...",2019-02-25 23:31:49.638,2019-02-25 10:50:21.024,2019-02-25 10:50:21.024,19210,51,...,2015-028A,DESCENDING,Sentinel-2A,02.11,Level-2A,S2MSI2A,Sentinel-2,1.11 GB,aee954a6-584d-43dd-b21a-6add2e7b1826,"POLYGON ((-0.49642944 44.22597512802537, 0.875..."
be59a9c4-2dda-45da-b403-78b53c13b5c0,S2A_MSIL2A_20190225T105021_N0211_R051_T31TCJ_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-02-25T10:50:21.024Z, Instrument: MS...",2019-02-25 23:31:44.968,2019-02-25 10:50:21.024,2019-02-25 10:50:21.024,19210,51,...,2015-028A,DESCENDING,Sentinel-2A,02.11,Level-2A,S2MSI2A,Sentinel-2,1.12 GB,be59a9c4-2dda-45da-b403-78b53c13b5c0,"POLYGON ((0.495928592903789 44.22596415476343,..."
dbaa503b-f16f-4ab2-94bb-bd910c72602f,S2A_MSIL2A_20190225T105021_N0211_R051_T30TXP_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-02-25T10:50:21.024Z, Instrument: MS...",2019-02-25 23:29:09.182,2019-02-25 10:50:21.024,2019-02-25 10:50:21.024,19210,51,...,2015-028A,DESCENDING,Sentinel-2A,02.11,Level-2A,S2MSI2A,Sentinel-2,453.22 MB,dbaa503b-f16f-4ab2-94bb-bd910c72602f,"POLYGON ((-1.0262451 43.2457798518931, -1.0190..."
e51cf778-bf56-4fe6-859a-0c3faff30471,S2A_MSIL2A_20190225T105021_N0211_R051_T31TCH_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-02-25T10:50:21.024Z, Instrument: MS...",2019-02-25 23:31:48.545,2019-02-25 10:50:21.024,2019-02-25 10:50:21.024,19210,51,...,2015-028A,DESCENDING,Sentinel-2A,02.11,Level-2A,S2MSI2A,Sentinel-2,1.16 GB,e51cf778-bf56-4fe6-859a-0c3faff30471,"POLYGON ((0.53321

Some columns need to be turned to string in order to be JSONizable.

In [ ]:
products_gdf['ingestiondate'] = products_gdf['ingestiondate'].astype(str)
products_gdf['beginposition'] = products_gdf['beginposition'].astype(str)
products_gdf['endposition'] = products_gdf['endposition'].astype(str)

**Print the footprint of the returned products**

In [ ]:
footprint_geojson = api.to_geojson(products)

m = folium.Map(
    location=[43.230775, 0.903595], 
    tiles='cartodbpositron',
    zoom_start=7
    )

folium.GeoJson(
    products_gdf,
    name='satellite imagery footprints',
    style_function=lambda x: {'fillColor': 'YlGnBu_09', 'color': 'YlGnBu_09',
                              'weight':5, 'fillOpacity':0.3},
    tooltip=folium.GeoJsonTooltip(fields=['title', 'ingestiondate'], 
                                  aliases=['Nom', 'Date'], 
                                  localize=True)
).add_to(m)

folium.GeoJson(
    area_of_interest,
    name='area_of_interest',
    style_function=lambda x: {'fillColor': 'blue', 'color': 'blue',
                              'weight':5, 'fillOpacity':0.3}
).add_to(m)

# Add a LayerControl.
folium.LayerControl().add_to(m)

m

## Download results
The area we are mainly interested in is the one surrounding Toulouse. We will then start the process to download product whose ID is "be59a9c4-2dda-45da-b403-78b53c13b5c0".

In [ ]:
sat_imagery_id = 'be59a9c4-2dda-45da-b403-78b53c13b5c0'

In [10]:
api.download(sat_imagery_id)

{'id': 'be59a9c4-2dda-45da-b403-78b53c13b5c0',
 'title': 'S2A_MSIL2A_20190225T105021_N0211_R051_T31TCJ_20190225T134315',
 'size': 1203064988,
 'md5': '55F68A959DD23344EE26CDE5D9F53F96',
 'date': datetime.datetime(2019, 2, 25, 10, 50, 21, 24000),
 'footprint': 'POLYGON((0.495928592903789 44.22596415476343,1.870237286761489 44.24783068396879,1.888683014192297 43.25939191053712,0.536772323136669 43.23826255332707,0.495928592903789 44.22596415476343))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('be59a9c4-2dda-45da-b403-78b53c13b5c0')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2019, 2, 25, 23, 43, 2, 874000),
 'Ingestion Date': datetime.datetime(2019, 2, 25, 23, 31, 44, 968000),
 'path': './S2A_MSIL2A_20190225T105021_N0211_R051_T31TCJ_20190225T134315.zip',
 'downloaded_bytes': 0}

In [ ]:
sat_imagery_zipfile = f"{products_gdf['title'][sat_imagery_id]}.zip"

In [ ]:
sat_imagery_zipfile = zipfile.ZipFile(sat_imagery_zipfile, 'r')
sat_imagery_zipfile.extractall()
sat_imagery_zipfile.close()

Folders naming convention:

`S2A_MSIL2A_20190225T105021_N0211_R051_T31TCJ_20190225T134315.SAFE`  

* `S2A` mission ID, here Sentinel2-A  
* `MSIL2A` File type, with instrument type (MSI : MultiSpectral Instrument) and product treatment level (here L2A)
* `20190225T105021` Data acquission timestamp, following UTC format `aaammjjThhmmss`  
  * aaaa = year
  * mm = month
  * jj = day
  * T = separator between day and hour
  * hh = hour
  * mm = minute
  * ss = second
* `N0211` N + production baseline number
* `R051` R + orbit number
* `T31TCJ` T + tile number (following US-MGRS naming convention (US Military Grid Reference System))
* `20190225T134315`  Data acquission timestamp, following UTC format `aaammjjThhmmss`

The extracted folder is named `<image_title>.SAFE`.  
It is composed of several folders and subfolders. We are particularly interested in the folder named `IMG_DATA` containing all bands in different spatial resolutions: 10m, 20m and 60m.  
The full path to `IMG_DATA` folder is: 

In [14]:
img_data_path = './S2A_MSIL2A_20190225T105021_N0211_R051_T31TCJ_20190225T134315.SAFE/GRANULE/L2A_T31TCJ_A019210_20190225T105315/IMG_DATA'

Files naming convention in folder `IMG_DATA` :

`T31TCJ_20190225T105021_B02_10m.jp2` 

* `T31TCJ` T + tile number
* `20190225T134315` Data acquission timestamp, following UTC format `aaammjjThhmmss`
* `B02` Bandwith (cf. list of available bandwiths, given spatial resolution, below)
* `10m` Resolution

* R10m:
    * `T31TCJ_20190225T105021_B02_10m.jp2`: blue
    * `T31TCJ_20190225T105021_B03_10m.jp2`: green
    * `T31TCJ_20190225T105021_B04_10m.jp2`: red
    * `T31TCJ_20190225T105021_TCI_10m.jp2`: true color image
    * `T31TCJ_20190225T105021_B08_10m.jp2`: NIR  = near infrared (vegetation discrimination)
    * `T31TCJ_20190225T105021_WVP_10m.jp2`: water vapour
    * `T31TCJ_20190225T105021_AOT_10m.jp2`: top-of-atmosphere
    
* R20m:
    * `T31TCJ_20190225T105021_B03_20m.jp2`: green
    * `T31TCJ_20190225T105021_B8A_20m.jp2`: NIR ~860nm (vegetation discrimination)
    * `T31TCJ_20190225T105021_SCL_20m.jp2`: scene classification map**
    * `T31TCJ_20190225T105021_TCI_20m.jp2`: true color image
    * `T31TCJ_20190225T105021_WVP_20m.jp2`: water vapour
    * `T31TCJ_20190225T105021_B12_20m.jp2`: SWIR ~2200nm (snow/ice/cloud discrimination)
    * `T31TCJ_20190225T105021_B04_20m.jp2`: red
    * `T31TCJ_20190225T105021_B02_20m.jp2`: blue
    * `T31TCJ_20190225T105021_B06_20m.jp2`: NIR ~750nm (vegetation discrimination)
    * `T31TCJ_20190225T105021_AOT_20m.jp2`: top-of-atmosphere
    * `T31TCJ_20190225T105021_B07_20m.jp2`: NIR ~775nm (vegetation discrimination)
    * `T31TCJ_20190225T105021_B05_20m.jp2`: NIR ~700nm (vegetation discrimination)
    * `T31TCJ_20190225T105021_B11_20m.jp2`: SWIR ~1600nm (snow/ice/cloud discrimination)

* R60m:
    * `T31TCJ_20190225T105021_B03_60m.jp2`: green
    * `T31TCJ_20190225T105021_B04_60m.jp2`: red
    * `T31TCJ_20190225T105021_B11_60m.jp2`: SWIR ~1600nm (snow/ice/cloud discrimination)
    * `T31TCJ_20190225T105021_B05_60m.jp2`: NIR ~700nm (vegetation discrimination)
    * `T31TCJ_20190225T105021_B12_60m.jp2`: SWIR ~2200nm (snow/ice/cloud discrimination)
    * `T31TCJ_20190225T105021_WVP_60m.jp2`: water vapour
    * `T31TCJ_20190225T105021_B01_60m.jp2`: blue ~450nm (aerosols discrimination)
    * `T31TCJ_20190225T105021_SCL_60m.jp2`: scene classification map
    * `T31TCJ_20190225T105021_AOT_60m.jp2`: top-of-atmosphere
    * `T31TCJ_20190225T105021_B07_60m.jp2`: NIR ~775nm (vegetation discrimination)
    * `T31TCJ_20190225T105021_B06_60m.jp2`: NIR ~750nm (vegetation discrimination)
    * `T31TCJ_20190225T105021_B09_60m.jp2`: NIR ~840nm (water vapour discrimination)
    * `T31TCJ_20190225T105021_TCI_60m.jp2`: true color image
    * `T31TCJ_20190225T105021_B02_60m.jp2`: blue
    * `T31TCJ_20190225T105021_B8A_60m.jp2`: NIR ~860nm (vegetation discrimination)
 
 
 
** scene classification map is a classification map which includes four different classes for clouds (including cirrus) and six different classifications for shadows, cloud shadows, vegetation, soils/deserts, water and snow.

    
 ![sentinel2_bands](media/sentinel2_bands.png)


[Toute la documentation sur la convention de nommage des produits Sentinel2](https://sentinel.esa.int/documents/247904/685211/Sentinel-2-Products-Specification-Document)